CARGAR LOS DATOS

In [4]:
import pandas as pd

orders  = pd.read_csv(r'data\orders.csv')
order_products_prior = pd.read_csv(r'data\order_products__prior.csv')
order_products_train = pd.read_csv(r'data\order_products__train.csv')
products = pd.read_csv(r'data\products.csv')
user_segmented = pd.read_csv(r'data\segmented_users.csv')

print(orders.columns)
print(order_products_prior.columns)
print(order_products_train.columns)
print(products.columns)
print(user_segmented.columns)

C:\Users\joant\AppData\Local\Temp\ipykernel_15144\411064258.py:3: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  orders  = pd.read_csv(r'data\orders.csv')


Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')
Index(['user_id', 'n_compras', 'last_order_date', 'mean_time_between_orders',
       'Segmento'],
      dtype='object')


HACER UN MERGE DE LOS CSV

In [5]:
order_products = pd.concat([order_products_prior, order_products_train], ignore_index=True)

merged_data = orders.merge(order_products, on="order_id", how="inner")

merged_data = merged_data.merge(products, on="product_id", how="left")

merged_data = merged_data.merge(user_segmented, on="user_id", how="left")

print(merged_data.head())

   order_id user_id eval_set order_number  order_dow  order_hour_of_day  \
0   2539329       1    prior            1          2                8.0   
1   2539329       1    prior            1          2                8.0   
2   2539329       1    prior            1          2                8.0   
3   2539329       1    prior            1          2                8.0   
4   2539329       1    prior            1          2                8.0   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     NaN         196                  1          0   
1                     NaN       14084                  2          0   
2                     NaN       12427                  3          0   
3                     NaN       26088                  4          0   
4                     NaN       26405                  5          0   

                              product_name  aisle_id  department_id  \
0                                     Soda        7

CREAR DATAFRAMES POR CADA SEGMENTO

In [8]:
df_frecuentes = merged_data.query("Segmento == 'Frecuente'")
df_nuevos = merged_data.query("Segmento == 'Nuevo'")
df_ocasionales = merged_data.query("Segmento == 'Ocasional'")
df_inactivos = merged_data.query("Segmento == 'Inactivo'")

# Mostrar un ejemplo
print(df_frecuentes.shape)
print(df_nuevos.shape)
print(df_ocasionales.shape)
print(df_inactivos.shape)

print(df_frecuentes.columns)


(7687378, 17)
(6961510, 17)
(6603693, 17)
(11283235, 17)
Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'n_compras', 'last_order_date',
       'mean_time_between_orders', 'Segmento'],
      dtype='object')


FILTRAR LOS PRODUCTOS MÁS POPULARES DE CADA SEGMENTO

In [ ]:
# Contar cuántas veces aparece cada producto
product_counts_frecuentes = df_frecuentes['product_id'].value_counts()
product_counts_nuevos = df_nuevos['product_id'].value_counts()
product_counts_ocasionales = df_ocasionales['product_id'].value_counts()
product_counts_inactivos= df_inactivos['product_id'].value_counts()

# Filtrar productos que aparecen en al menos 500 compras (puedes ajustar este número)
popular_products_frecuentes = product_counts_frecuentes[product_counts_frecuentes > 500].index
popular_products_nuevos = product_counts_nuevos[product_counts_nuevos > 500].index
popular_products_ocasionales = product_counts_ocasionales[product_counts_ocasionales > 500].index
popular_products_inactivos= product_counts_inactivos[product_counts_inactivos > 500].index

# Filtrar el dataset para trabajar solo con los productos más comprados
order_products_filtered_frecuentes = df_frecuentes[df_frecuentes['product_id'].isin(popular_products_frecuentes)]
order_products_filtered_nuevos = df_nuevos[df_nuevos['product_id'].isin(popular_products_nuevos)]
order_products_filtered_ocasionales = df_ocasionales[df_ocasionales['product_id'].isin(popular_products_ocasionales)]
order_products_filtered_inactivos = df_inactivos[df_inactivos['product_id'].isin(popular_products_inactivos)]

LIMITAR EL NÚMERO DE TRANSACCIONES

In [ ]:
# Tomar una muestra de 500,000 órdenes para reducir memoria
sample_orders_frecuentes = order_products_filtered_frecuentes['order_id'].sample(n=100000, random_state=42)
sample_orders_nuevos = order_products_filtered_nuevos['order_id'].sample(n=100000, random_state=42)
sample_orders_ocasionales = order_products_filtered_ocasionales['order_id'].sample(n=100000, random_state=42)
sample_orders_inactivos = order_products_filtered_inactivos['order_id'].sample(n=100000, random_state=42)

# Filtrar las órdenes en la muestra
order_sample_frecuentes = order_products_filtered_frecuentes[order_products_filtered_frecuentes['order_id'].isin(sample_orders_frecuentes)]
order_sample_nuevos = order_products_filtered_nuevos[order_products_filtered_nuevos['order_id'].isin(sample_orders_nuevos)]
order_sample_ocasionales = order_products_filtered_ocasionales[order_products_filtered_ocasionales['order_id'].isin(sample_orders_ocasionales)]
order_sample_inactivos = order_products_filtered_inactivos[order_products_filtered_inactivos['order_id'].isin(sample_orders_inactivos)]

CREAR LA MATRIZ DE TRANSACCIONES CON One-Hot Encoding

In [ ]:
from scipy.sparse import coo_matrix

# Eliminar duplicados
order_sample_filtered_frecuentes = order_sample_frecuentes.drop_duplicates(subset=['order_id', 'product_name'])
order_sample_filtered_nuevos = order_sample_nuevos.drop_duplicates(subset=['order_id', 'product_name'])
order_sample_filtered_ocasionales = order_sample_ocasionales.drop_duplicates(subset=['order_id', 'product_name'])
order_sample_filtered_inactivos = order_sample_inactivos.drop_duplicates(subset=['order_id', 'product_name'])

# Crear la matriz dispersa (sparse matrix)
product_to_id_frecuentes = {product: idx for idx, product in enumerate(order_sample_filtered_frecuentes['product_name'].unique())}
product_to_id_nuevos = {product: idx for idx, product in enumerate(order_sample_filtered_nuevos['product_name'].unique())}
product_to_id_ocasionales = {product: idx for idx, product in enumerate(order_sample_filtered_ocasionales['product_name'].unique())}
product_to_id_inactivo = {product: idx for idx, product in enumerate(order_sample_filtered_inactivos['product_name'].unique())}

order_sample_filtered_frecuentes['product_id_num'] = order_sample_filtered_frecuentes['product_name'].map(product_to_id_frecuentes)
order_sample_filtered_nuevos['product_id_num'] = order_sample_filtered_nuevos['product_name'].map(product_to_id_nuevos)
order_sample_filtered_ocasionales['product_id_num'] = order_sample_filtered_ocasionales['product_name'].map(product_to_id_ocasionales)
order_sample_filtered_inactivos['product_id_num'] = order_sample_filtered_inactivos['product_name'].map(product_to_id_inactivo)

row_frecuentes = order_sample_filtered_frecuentes['order_id'].astype('category').cat.codes
row_nuevos = order_sample_filtered_nuevos['order_id'].astype('category').cat.codes
row_ocasionales = order_sample_filtered_ocasionales['order_id'].astype('category').cat.codes
row_inactivos = order_sample_filtered_inactivos['order_id'].astype('category').cat.codes

col_frecuentes = order_sample_filtered_frecuentes['product_id_num']
col_nuevos = order_sample_filtered_nuevos['product_id_num']
col_ocasionales = order_sample_filtered_ocasionales['product_id_num']
col_inactivos = order_sample_filtered_inactivos['product_id_num']

data_frecuentes = [1] * len(order_sample_filtered_frecuentes)
data_nuevos = [1] * len(order_sample_filtered_nuevos)
data_ocasionales = [1] * len(order_sample_filtered_ocasionales)          
data_inactivos = [1] * len(order_sample_filtered_inactivos)

# Crear la matriz dispersa
basket_sparse_frecuentes = coo_matrix((data_frecuentes, (row_frecuentes, col_frecuentes)), shape=(order_sample_filtered_frecuentes['order_id'].nunique(), len(product_to_id_frecuentes)))
basket_sparse_nuevos = coo_matrix((data_nuevos, (row_nuevos, col_nuevos)), shape=(order_sample_filtered_nuevos['order_id'].nunique(), len(product_to_id_nuevos)))
basket_sparse_ocasionales = coo_matrix((data_ocasionales, (row_ocasionales, col_ocasionales)), shape=(order_sample_filtered_ocasionales['order_id'].nunique(), len(product_to_id_ocasionales)))
basket_sparse_inactivos = coo_matrix((data_inactivos, (row_inactivos, col_inactivos)), shape=(order_sample_filtered_inactivos['order_id'].nunique(), len(product_to_id_inactivo)))

APLICAR EL ALGORITMO APRIORI

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

basket_df = pd.DataFrame.sparse.from_spmatrix(basket_sparse, columns=product_to_id.keys())

# Convertir los valores de la matriz en binarios (1 si el producto fue comprado, 0 si no)
basket_df = basket_df.applymap(lambda x: 1 if x > 0 else 0)

# Aplicar Apriori para obtener los conjuntos frecuentes
frequent_itemsets = apriori(basket_df, min_support=0.01, use_colnames=True)

# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

C:\Users\joant\AppData\Local\Temp\ipykernel_4572\691805405.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_df = basket_df.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\joant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


FILTRAR REGLAS POR LAS MAS CONFIABLES

In [12]:
confidence = 0.25

rules_sorted = rules.sort_values(by=['support', 'confidence', 'lift'], ascending=[False, False, False])

rules_filtered = rules_sorted[rules_sorted['confidence'] > confidence]


rules_filtered.to_csv(f'filtered_rules_{confidence}.csv',index=False)
rules_sorted.to_csv('sorted_rules.csv',index=False)